In [1]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver  # Import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.parse import urljoin
from selenium.common.exceptions import TimeoutException

def setup_driver():  
    """Configure and return a headless Chrome driver"""  
    options = webdriver.ChromeOptions()  
    options.add_argument("--headless")  
    options.add_argument("--disable-gpu")  
    options.add_argument("--no-sandbox")  
    options.add_argument("--disable-dev-shm-usage")  
    options.add_argument("--window-size=1920x1080")  
    
    return webdriver.Chrome(options=options)  

driver = setup_driver()

In [2]:
urls = 'https://wan-ifra.org/news/'

def get_news_urls_titles(url):
    driver.get(url) 
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    posts = soup.find_all('div', class_='post')
    for post in posts:
        title  = post.find('h1').text
        link = post.find('a')['href']
        print(title, link)
    # content  = 


def get_article_content(url):
    driver.get(url)
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    article_info = soup.find('p', class_='info').text

    content = soup.find('div', class_='content-area')
    content = content.text
    content = content.replace(article_info, ' ')
    print(article_info)
    print(content)



    


In [15]:
get_news_urls_titles('https://wan-ifra.org/news/')

Accelerating Action: why, and how, WAN-IFRA WIN will keep driving inclusivity in global news media https://wan-ifra.org/2025/03/accelerating-action-why-and-how-wan-ifra-win-is-still-driving-inclusivity-in-global-news-media/
‘Just keep swimming’ – KBR’s Citra Prastuti on leading from the front in challenging times https://wan-ifra.org/2025/03/just-keep-swimming-kbrs-citra-prastuti-on-leading-from-the-front-in-challenging-times/
Who is Setting the News Agenda? New report shows some progress in gender equality – but barriers remain https://wan-ifra.org/2025/03/who-is-setting-the-news-agenda-new-report-shows-some-progress-in-gender-equality-but-barriers-remain/
How India’s Amar Ujala used an AI chatbot to boost user experience at Maha Kumbh https://wan-ifra.org/2025/03/how-indias-amar-ujala-used-an-ai-chatbot-to-boost-pilgrim-experience-at-maha-kumbh/
Poland’s top editor warns: ‘Disinformation is demolishing democracy and civil society’ https://wan-ifra.org/2025/03/polands-top-editor-warns

In [23]:
get_article_content('https://wan-ifra.org/2025/02/lab-to-launch-the-hindus-ai-integration-strategy/')

by Neha Gupta neha.gupta@wan-ifra.org | February 25, 2025
 Speaking at our recent Bangalore AI Forum, Suresh Vijayaraghavan, Chief Technology Officer at The Hindu, detailed the publication’s AI journey – highlighting the challenges of moving from standalone experiments to full-scale integration within editorial and production workflows.“AI in isolation is innovation, but AI in production is transformation,” Vijayaraghavan said, underscoring the need for seamless AI integration rather than relying on fragmented tools.From siloed systems to an integrated platformFor over a decade, The Hindu has adopted various digital tools, including e-paper platforms, paywalls, SEO systems, social media automation, and recommendation engines. However, these systems operated independently, creating inefficiencies in editorial workflows.By 2020, the company launched a three-year digital transformation to unify its CMS, analytics, and AI-powered tools into a single integrated platform. The system, complet

In [1]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.parse import urljoin
from selenium.common.exceptions import TimeoutException

def setup_driver():  
    """Configure and return a headless Chrome driver"""  
    options = webdriver.ChromeOptions()  
    options.add_argument("--headless")  
    options.add_argument("--disable-gpu")  
    options.add_argument("--no-sandbox")  
    options.add_argument("--disable-dev-shm-usage")  
    options.add_argument("--window-size=1920x1080")  
    
    return webdriver.Chrome(options=options)  

driver = setup_driver()

def scrape_all_news_pages():
    page_num = 1
    extracted_data = []  # List to store extracted data
    
    while True:
        if page_num == 1:
            url = 'https://wan-ifra.org/news/'
        else:
            url = f'https://wan-ifra.org/news/page/{page_num}/'
        
        driver.get(url)
        time.sleep(2)  
        
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        posts = soup.find_all('div', class_='post')
        
        # Stop if no posts found
        if not posts:
            print(f"No more posts on page {page_num}. Exiting.")
            break
        
        print(f"Scraping page {page_num}...")
        for post in posts:
            # Extract title and link with error handling
            title_element = post.find('h1')
            link_element = post.find('a')
            
            if title_element and link_element:
                title = title_element.get_text(strip=True)
                link = link_element.get('href')
                article_info, content = get_article_content(link)

                
                # Add the extracted data to the list
                extracted_data.append([title, link, article_info, content])
                print(f"Title: {title}\nLink: {link}\nArticleInfo {article_info}\n")
        
        page_num += 1
    
    # Save the extracted data to CSV 
    save_to_csv(extracted_data)

def get_article_content(url):
    driver.get(url)
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # Handle article_info extraction safely
    article_info_element = soup.find('p', class_='info')
    article_info = article_info_element.text.strip() if article_info_element else ''
    
    # Handle content extraction safely
    content_element = soup.find('div', class_='content-area')
    content = content_element.text.strip() if content_element else ''
    
    # Remove article_info from content (if both exist)
    if article_info and content:
        content = content.replace(article_info, ' ').strip()
    
    return article_info, content


def save_to_csv(data):
    # Create a DataFrame from the extracted data
    df = pd.DataFrame(data, columns=['Title', 'Link','article_info', 'content'])
    
    # exporte DataFrame to a CSV file
    df.to_csv('wan_ifra_news.csv', index=False)
    print("Data has been saved to 'scraped_news_data.csv'.")

# Start scraping
scrape_all_news_pages()

driver.quit()


Scraping page 1...
Title: Gremi Media targets professional audiences to drive digital growth
Link: https://wan-ifra.org/2025/03/gremi-media-targets-professional-audiences-to-transform-its-audience-strategy-and-drive-digital-growth/
ArticleInfo by WAN-IFRA External Contributor info@wan-ifra.org | March 14, 2025

Title: WAN-IFRA President: Why you should join us in Krakow for the World News Media Congress
Link: https://wan-ifra.org/2025/03/wan-ifra-president-why-you-should-join-us-in-krakow-for-the-world-news-media-congress/
ArticleInfo by WAN-IFRA Staff executivenews@wan-ifra.org | March 13, 2025

Title: WAN-IFRA, MBL and AIRPPU Unite to Strengthen Independent Journalism in Ukraine
Link: https://wan-ifra.org/2025/03/wan-ifra-mbl-and-airppu-unite-to-strengthen-independent-journalism-in-ukraine/
ArticleInfo by WAN-IFRA Staff executivenews@wan-ifra.org | March 12, 2025

Title: Accelerating Action: why, and how, WAN-IFRA WIN will keep driving inclusivity in global news media
Link: https://w

ReadTimeoutError: HTTPConnectionPool(host='localhost', port=58048): Read timed out. (read timeout=120)

In [ ]:


url = 'https://wan-ifra.org/2025/03/polands-top-editor-warns-disinformation-is-demolishing-democracy-and-civil-society/'
get_article_content(url)

('by Cherilyn Ireton cherilyn.ireton@wan-ifra.org | March 3, 2025',
 'Founded in 1989 as the voice of Lech Wałęsa’s Solidarity movement, Gazeta Wyborcza was originally launched to cover Poland’s first partially free elections. Owned by Agora, today it stands as the country’s largest national and digital newspaper, one of Europe’s most influential news brands, and a beacon of independent journalism. Over the past decade, it has been at the forefront of a fierce battle for democracy and independent journalism, resisting sustained interference from the conservative Law and Justice party, which took full control of public media during its eight-year rule.Donald Tusk’s election as Prime Minister last year eased some of the pressure on the press, but Poland’s political future remains uncertain, with a presidential election looming – just two weeks after our Congress takes place. Like much of Europe, the country is also navigating a shifting geopolitical landscape, particularly in relation to